# Example Predictor: SVR Predictor

This example contains basic functionality for training and evaluating a SVR predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import time

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


### Copy the data locally

In [2]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [3]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7f65b6ac8280>)

In [4]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [5]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'M1_Wildcard', 'ConfirmedCases',
       'ConfirmedDeaths', 'StringencyIndex', 'StringencyIndexForDisplay',
       'StringencyLegacyIndex', 'StringencyLegacyIndexForDisplay',
       'GovernmentResponseIndex', 'Gove

In [6]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [7]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [8]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [9]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [10]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [11]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [12]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87064,Zimbabwe,NaN,Zimbabwe__nan,2020-07-27,78.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
87065,Zimbabwe,NaN,Zimbabwe__nan,2020-07-28,192.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
87066,Zimbabwe,NaN,Zimbabwe__nan,2020-07-29,113.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
87067,Zimbabwe,NaN,Zimbabwe__nan,2020-07-30,62.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0


In [13]:
# Set number of past days to use to make predictions
nb_lookback_days = 2

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d + 1]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [14]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [15]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.5,
                                                    random_state=301)
print(X_train.shape)

(27825, 26)


In [16]:
# Create and train SVR model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
start_time = time.time()  
  


model = SVR(kernel='poly')
# Fit model
model.fit(X_train, y_train)

interval = time.time() - start_time
print ('Total time in seconds:', interval)

Total time in seconds: 151.36163139343262


In [17]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 382.24827551287336
Test MAE: 378.11705666028627


In [195]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Day -7 NewCases 0.0013246240389721805
Day -6 NewCases 0.43932558850808323
Day -5 NewCases 0.21733329048235964
Day -4 NewCases 0.05883044887710127
Day -3 NewCases 0.06951319628060729
Day -2 NewCases 0.052031127397790256
Day -1 NewCases 0.23822431735053784
Day -26 C6_Stay at home requirements 4.314491844181091
Day -22 C2_Workplace closing 9.715799339190195
Day -17 C2_Workplace closing 5.77128206222885
Intercept 26.55971959613379


In [196]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [197]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [198]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="../../../validation/data/2020-09-30_historical_ip.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 58.844691538124245
2020-08-02: 71.32339797888373
2020-08-03: 78.99821175219239
2020-08-04: 90.24702060691985
2020-08-05: 87.81468190769263
2020-08-06: 98.77662008875717
2020-08-07: 131.04966110524353
2020-08-08: 147.02909378415967
2020-08-09: 158.9669020318663
2020-08-10: 169.95393176817132
2020-08-11: 177.53057188557548
2020-08-12: 193.50373596073152
2020-08-13: 216.8349592365735
2020-08-14: 234.05458053144855
2020-08-15: 248.58034443852705
2020-08-16: 261.9875797943422
2020-08-17: 275.32154428012313
2020-08-18: 293.3164901988505
2020-08-19: 314.09720089504225
2020-08-20: 332.4524664281599
2020-08-21: 349.2598980364087
2020-08-22: 365.52936360035164
2020-08-23: 382.5647394069261
2020-08-24: 407.9286106098403
2020-08-25: 430.1195473706485
2020-08-26: 450.6112812074667
2020-08-27: 466.04250578239487
2020-08-28: 484.6917479475174
2020-08-29: 515.4011439840982
2020-08-30: 541.9538140452253
2020-08-31: 566.3175244293836
(31,)

Predicting for Afghanist

2020-08-01: 2144.332695101401
2020-08-02: 2777.197014257926
2020-08-03: 3001.935197889325
2020-08-04: 2959.720273647507
2020-08-05: 2638.85852292054
2020-08-06: 1675.639874058618
2020-08-07: 2515.1391339682514
2020-08-08: 2969.940284086295
2020-08-09: 3126.5290564056013
2020-08-10: 3101.2034748194906
2020-08-11: 2833.9624209230765
2020-08-12: 2565.4820421577147
2020-08-13: 2961.4513938292084
2020-08-14: 3256.694647335901
2020-08-15: 3377.1384643214483
2020-08-16: 3363.9240914269913
2020-08-17: 3235.0721866850313
2020-08-18: 3208.726290666877
2020-08-19: 3439.682958890746
2020-08-20: 3640.0062092948474
2020-08-21: 3740.766425811254
2020-08-22: 3756.0479216754384
2020-08-23: 3749.5237225945566
2020-08-24: 3806.0033506906666
2020-08-25: 3971.07593144318
2020-08-26: 4123.996042396558
2020-08-27: 4218.95278380375
2020-08-28: 4269.756934972487
2020-08-29: 4316.5705832789645
2020-08-30: 4406.643306209051
2020-08-31: 4545.484546546078
(31,)

Predicting for Burundi__nan
2020-08-01: 56.720283064

2020-08-17: 1805.3405240528039
2020-08-18: 1794.6481259781447
2020-08-19: 1922.5576722084572
2020-08-20: 2027.8611327454012
2020-08-21: 2086.480804507241
2020-08-22: 2103.064322605483
2020-08-23: 2084.465891235773
2020-08-24: 2114.18393196926
2020-08-25: 2203.9629103789403
2020-08-26: 2285.751431772877
2020-08-27: 2331.7461660028644
2020-08-28: 2358.2690831540385
2020-08-29: 2376.2576837027927
2020-08-30: 2422.4751400840555
2020-08-31: 2496.227749088482
(31,)

Predicting for Belize__nan
2020-08-01: 90.24670752412706
2020-08-02: 115.09458860912723
2020-08-03: 115.59422268420562
2020-08-04: 130.1142254691766
2020-08-05: 133.13879965417544
2020-08-06: 123.67937584586683
2020-08-07: 167.5732706686711
2020-08-08: 189.74658547057885
2020-08-09: 200.24113547699562
2020-08-10: 213.42660908892782
2020-08-11: 220.5295470030658
2020-08-12: 230.32951445957664
2020-08-13: 258.6578228489191
2020-08-14: 279.26593594832696
2020-08-15: 294.2538775634252
2020-08-16: 308.79268298359665
2020-08-17: 321.40

2020-08-01: 52.43243951479994
2020-08-02: 67.11107907783762
2020-08-03: 71.15796934836747
2020-08-04: 74.76925341890174
2020-08-05: 79.93979797389343
2020-08-06: 93.89962444224932
2020-08-07: 124.2084054878473
2020-08-08: 140.12459342605703
2020-08-09: 149.34999247323088
2020-08-10: 158.01958873847354
2020-08-11: 168.76464073775372
2020-08-12: 186.0799708599353
2020-08-13: 208.70239852942325
2020-08-14: 225.28702891751902
2020-08-15: 238.20897047437683
2020-08-16: 250.897687269757
2020-08-17: 265.5717875156541
2020-08-18: 284.1395573739485
2020-08-19: 304.5345384813852
2020-08-20: 322.2501342423749
2020-08-21: 338.1421712966197
2020-08-22: 354.14063913468317
2020-08-23: 371.7089674232608
2020-08-24: 391.4827539364624
2020-08-25: 411.9893120773748
2020-08-26: 431.32955191499707
2020-08-27: 445.5796912783702
2020-08-28: 463.4372624886415
2020-08-29: 483.02054313247726
2020-08-30: 503.91043366777944
2020-08-31: 525.2240592171854
(31,)

Predicting for Canada__nan
2020-08-01: 3992.723776106

2020-08-16: 8403.039349827659
2020-08-17: 8067.457024749086
2020-08-18: 8025.8694388755885
2020-08-19: 8630.641021762716
2020-08-20: 9086.552259317094
2020-08-21: 9274.101001248462
2020-08-22: 9272.666120133108
2020-08-23: 9212.2059980136
2020-08-24: 9350.308830521859
2020-08-25: 9755.717541779833
2020-08-26: 10097.049363235892
2020-08-27: 10288.186873849114
2020-08-28: 10374.264317567733
2020-08-29: 10455.743363970292
2020-08-30: 10661.700474625331
2020-08-31: 10984.704260040297
(31,)

Predicting for Comoros__nan
2020-08-01: 51.18706624998747
2020-08-02: 63.54443598451152
2020-08-03: 69.34759312089086
2020-08-04: 75.8178352984691
2020-08-05: 79.77950362676793
2020-08-06: 93.30966400646543
2020-08-07: 122.7036326801595
2020-08-08: 137.82394736496258
2020-08-09: 148.10101007372515
2020-08-10: 157.88646983143383
2020-08-11: 168.2222404193158
2020-08-12: 185.13522851724812
2020-08-13: 207.20451086000236
2020-08-14: 223.5513328183471
2020-08-15: 237.03927836889264
2020-08-16: 250.191006745

...

2020-08-11: 2170.6367135288497
2020-08-12: 2155.570032561928
2020-08-13: 2586.958296836104
2020-08-14: 2789.5935218660125
2020-08-15: 2817.820621666483
2020-08-16: 2721.975554083676
2020-08-17: 2617.1650375613526
2020-08-18: 2687.8688627026418
2020-08-19: 2927.796265412865
2020-08-20: 3071.4362397489226
2020-08-21: 3108.726245784462
2020-08-22: 3081.0724253690328
2020-08-23: 3069.7181588963585
2020-08-24: 3159.6843657077043
2020-08-25: 3317.514862620432
2020-08-26: 3428.9056997280386
2020-08-27: 3475.487801729189
2020-08-28: 3494.077340873483
2020-08-29: 3532.485713576316
2020-08-30: 3626.205238222311
2020-08-31: 3748.229687689558
(31,)

Predicting for United States__Maine
2020-08-01: 229.75798158881733
2020-08-02: 266.7073691764357
2020-08-03: 261.1262599357003
2020-08-04: 286.8367670697391
2020-08-05: 246.23826344996743
2020-08-06: 208.2815160805459
2020-08-07: 307.30962731708826
2020-08-08: 342.94022163834734
2020-08-09: 354.7386859580012
2020-08-10: 366.5184995324431
2020-08-11: 35

2020-08-30: 2899.2467968074798
2020-08-31: 2987.6755023912033
(31,)

Predicting for United States__New Hampshire
2020-08-01: 472.7462842270983
2020-08-02: 552.5428458773952
2020-08-03: 520.3489103820864
2020-08-04: 475.0762745142402
2020-08-05: 453.072157550527
2020-08-06: 355.33776709215755
2020-08-07: 550.3116614793696
2020-08-08: 616.1670364240074
2020-08-09: 610.0345876206968
2020-08-10: 595.0894197224493
2020-08-11: 576.2903808799631
2020-08-12: 573.8903461390325
2020-08-13: 670.7809281159223
2020-08-14: 719.4091653230694
2020-08-15: 728.907139587699
2020-08-16: 729.634374109035
2020-08-17: 730.5819270137977
2020-08-18: 754.3448022260689
2020-08-19: 813.7961892898986
2020-08-20: 852.8602140636394
2020-08-21: 871.3622853010609
2020-08-22: 883.8711191641103
2020-08-23: 899.6084396176636
2020-08-24: 930.9542528528277
2020-08-25: 975.8383380463497
2020-08-26: 1011.253320792458
2020-08-27: 1031.7144543910101
2020-08-28: 1052.335802665084
2020-08-29: 1077.1581504608457
2020-08-30: 1111.

2020-08-14: 1147.6394667262512
2020-08-15: 1153.5166431511766
2020-08-16: 1300.4150011145573
2020-08-17: 1377.0190924562787
2020-08-18: 1313.807808148768
2020-08-19: 1344.4825837736046
2020-08-20: 1367.940889482997
2020-08-21: 1409.7581853405115
2020-08-22: 1500.5468095094936
2020-08-23: 1547.896747394844
2020-08-24: 1547.1851919077053
2020-08-25: 1576.7412439119137
2020-08-26: 1611.812529243038
2020-08-27: 1658.2607326936193
2020-08-28: 1723.3951730263734
2020-08-29: 1766.2729013752285
2020-08-30: 1791.3420902142482
2020-08-31: 1827.4113741918932
(31,)

Predicting for United States__South Carolina
2020-08-01: 1503.5932477978881
2020-08-02: 1856.3203346654025
2020-08-03: 1741.0927577721152
2020-08-04: 1503.2189802361167
2020-08-05: 1270.7897543042816
2020-08-06: 990.3059863383264
2020-08-07: 1623.641395341552
2020-08-08: 1861.6832269702911
2020-08-09: 1816.319413208519
2020-08-10: 1690.2511381345223
2020-08-11: 1548.2790739469183
2020-08-12: 1532.8624161569971
2020-08-13: 1839.97397482

2020-08-31: 4035.1098690478548
(31,)

Predicting for United States__Wisconsin
2020-08-01: 5744.773321377702
2020-08-02: 6542.4368432057445
2020-08-03: 6082.265989289753
2020-08-04: 4765.214209486694
2020-08-05: 3818.279705263088
2020-08-06: 3269.0216393823684
2020-08-07: 5662.888587086608
2020-08-08: 6319.313413807025
2020-08-09: 6019.008676515706
2020-08-10: 5330.691060396875
2020-08-11: 4800.425107464653
2020-08-12: 4933.741071779466
2020-08-13: 6066.018033689139
2020-08-14: 6491.624984634177
2020-08-15: 6349.343619851815
2020-08-16: 6006.107000023757
2020-08-17: 5808.237125499828
2020-08-18: 6062.337126276415
2020-08-19: 6670.457977046128
2020-08-20: 6952.158317555319
2020-08-21: 6920.38911290193
2020-08-22: 6790.7145330456915
2020-08-23: 6781.368251660932
2020-08-24: 7030.2942387931735
2020-08-25: 7406.947573049391
2020-08-26: 7619.008146187526
2020-08-27: 7659.796651689267
2020-08-28: 7662.331319530042
2020-08-29: 7751.779177176103
2020-08-30: 7979.737155711312
2020-08-31: 8253.16

2020-08-07: 2521.6574016022614
2020-08-08: 2865.719893133229
2020-08-09: 2965.390964640279
2020-08-10: 2838.9535859935927
2020-08-11: 2531.316068042638
2020-08-12: 2422.9349242807853
2020-08-13: 2864.1281770361966
2020-08-14: 3108.8280871521965
2020-08-15: 3181.2097195975625
2020-08-16: 3113.2024154740207
2020-08-17: 2984.8576435840996
2020-08-18: 3023.806734990132
2020-08-19: 3272.803629162292
2020-08-20: 3445.0430394452064
2020-08-21: 3511.5301272700904
2020-08-22: 3498.255679540122
2020-08-23: 3486.879799399192
2020-08-24: 3569.290393708765
2020-08-25: 3738.195075101608
2020-08-26: 3871.5973432775745
2020-08-27: 3943.7773362709063
2020-08-28: 3976.2867011351263
2020-08-29: 4019.8920518158193
2020-08-30: 4117.73560722296
2020-08-31: 4253.1254337314385
(31,)

Predicting for Zambia__nan
2020-08-01: 86.5921520086615
2020-08-02: 110.42544426703182
2020-08-03: 98.01342284222659
2020-08-04: 101.56612547728474
2020-08-05: 105.94891999794973
2020-08-06: 113.3686946414708
2020-08-07: 158.0632

In [199]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
213,Aruba,NaN,2020-08-01,58.844692
214,Aruba,NaN,2020-08-02,71.323398
215,Aruba,NaN,2020-08-03,78.998212
216,Aruba,NaN,2020-08-04,90.247021
217,Aruba,NaN,2020-08-05,87.814682


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [200]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
/home/cdrutinus/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Lasso from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [201]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,58.844691538124245
Aruba,,2020-08-02,71.32339797888373
Aruba,,2020-08-03,78.99821175219239
Aruba,,2020-08-04,90.24702060691985
Afghanistan,,2020-08-01,149.93664407981095
Afghanistan,,2020-08-02,287.2826933086321
Afghanistan,,2020-08-03,277.50656117987995
Afghanistan,,2020-08-04,264.8862561764254
Angola,,2020-08-01,198.70750682167088


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [202]:
import os
print(os.getcwd())

/home/cdrutinus/Cours/SDD/Mini-hackaton/pangolin/covid-xprize/covid_xprize/examples/predictors/linear


In [203]:
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [204]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
/home/cdrutinus/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Lasso from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
Saved predictions to predictions/val_4_days.csv
Done!
All good!


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [205]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
/home/cdrutinus/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Lasso from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
Saved predictions to predictions/val_1_month_future.csv
Done!
All good!
CPU times: user 62.5 ms, sys: 109 ms, total: 172 ms
Wall time: 4.08 s


## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [56]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-01
End date: 2021-05-30


In [57]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


### Check it

In [58]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-01 to 2021-05-30...
Saved predictions to predictions/val_6_month_future.csv
Done!
All good!
CPU times: user 7.59 s, sys: 277 ms, total: 7.87 s
Wall time: 2min 49s
